In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-06-01"
end_date = "2019-07-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-06-01** to **2019-07-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 30 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                         repo  author
  50          nteract                    bookstore      20
  51          nteract                     hydrogen      20
  52          nteract                      nteract      12
  53          nteract                    papermill      11
  54          nteract                    scrapbook       2
  55          nteract                         vdom       1
  39       jupyterlab                   jupyterlab     554
  43       jupyterlab               jupyterlab-git      35
  49       jupyterlab            jupyterlab_server      20
  41       jupyterlab     jupyterlab-data-explorer      15
  42       jupyterlab              jupyterlab-demo      15
  40       jupyterlab        jupyterlab-commenting      10
  44       jupyterlab            jupyterlab-github       5
  45       jupyterlab             jupyterlab-latex       5
  46       jupyterlab  jupyterlab-metadata-service       5
  47       jupyterlab         jupyterlab-telemetry       5
  48       jupyterlab               jupyterlab-toc       5
  29       jupyterhub          mybinder.org-deploy      80
  35       jupyterhub      the-littlest-jupyterhub      28
  25       jupyterhub                    binderhub      26
  24       jupyterhub                 batchspawner      14
  23       jupyterhub         alabaster-jupyterhub      12
  27       jupyterhub                   jupyterhub      12
  34       jupyterhub                 team-compass      10
  38       jupyterhub       zero-to-jupyterhub-k8s       8
  31       jupyterhub               oauthenticator       6
  30       jupyterhub                  nbgitpuller       4
  36       jupyterhub                traefik-proxy       4
  37       jupyterhub                  wrapspawner       4
  26       jupyterhub                   chartpress       2
  28       jupyterhub        kerberosauthenticator       2
  32       jupyterhub                  sudospawner       2
  33       jupyterhub               systemdspawner       2
  21  jupyter-widgets                     tutorial      64
  17  jupyter-widgets                   ipyleaflet      48
  18  jupyter-widgets                   ipywidgets      24
  19  jupyter-widgets                    pythreejs       8
  20  jupyter-widgets                   traittypes       4
  22  jupyter-widgets          widget-cookiecutter       4
  14          jupyter                     notebook      32
  12          jupyter                     nbgrader      24
  3           jupyter           enterprise_gateway      11
  15          jupyter                  repo2docker      11
  10          jupyter                    nbconvert       9
  7           jupyter                 jupyter_core       7
  4           jupyter                 jupyter-book       6
  6           jupyter               jupyter_client       5
  16          jupyter                    telemetry       4
  5           jupyter               jupyter-sphinx       2
  8           jupyter               jupyter_server       2
  13          jupyter                     nbviewer       2
  2           jupyter                docker-stacks       1
  9           jupyter                        nb2kg       1
  11          jupyter                       nbdime       1
  1           ipython                      ipython      84
  0           ipython                    ipykernel       9,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['bookstore', 'hydrogen', 'nteract', 'papermill', 'scrapbook', 'vdom', 'jupyterlab', 'jupyterlab-git', 'jupyterlab_server', 'jupyterlab-data-explorer', 'jupyterlab-demo', 'jupyterlab-commenting', 'jupyterlab-github', 'jupyterlab-latex', 'jupyterlab-metadata-service', 'jupyterlab-telemetry', 'jupyterlab-toc', 'mybinder.org-deploy', 'the-littlest-jupyterhub', 'binderhub', 'batchspawner', 'alabaster-jupyterhub', 'jupyterhub', 'team-compass', 'zero-to-jupyterhub-k8s', 'oauthenti

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:           username  authored authorAssociation
    0          Carreau        60            MEMBER
    2   LucianaMarques         6       CONTRIBUTOR
    1          Chilipp         3       CONTRIBUTOR
    3             alok         3       CONTRIBUTOR
    4          anntzer         3       CONTRIBUTOR
    5          blueyed         3       CONTRIBUTOR
    6           daharn         3       CONTRIBUTOR
    7         eobrie16         3       CONTRIBUTOR
    8          hroncok         3       CONTRIBUTOR
    9           kmaork         3       CONTRIBUTOR
    10        steff456         3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:          username  authored authorAssociation
    27       jhamrick        19            MEMBER
    29  joshuazeltser        14       CONTRIBUTOR
    30    kevin-bates        12            MEMBER
    43      takluyver         5            MEMBER
    14      choldgraf         4            MEMBER
    42       t-makaro         4      COLLABORATOR
    12        betatim         3            MEMBER
    2           MSeal         3            MEMBER
    6         Zsailer         3            MEMBER
    35          minrk         3            MEMBER
    17   danielmaitre         2       CONTRIBUTOR
    31       krinsman         2       CONTRIBUTOR
    44    thammegowda         2       CONTRIBUTOR
    25     jaipreet-s         2      COLLABORATOR
    32       lresende         2            MEMBER
    19       echarles         2       CONTRIBUTOR
    37        nthiery         1       CONTRIBUTOR
    34         mgeier         1       CONTRIBUTOR
    33      marwahaha         1       CONTRIBUTOR
    36       mtd91429         1       CONTRIBUTOR
    0           BertR         1            MEMBER
    38        rkdarst         1       CONTRIBUTOR
    39       saraedum         1       CONTRIBUTOR
    40      sigurdurb         1       CONTRIBUTOR
    45         tholoz         1       CONTRIBUTOR
    46         tmetzl         1       CONTRIBUTOR
    47       tmorrell         1       CONTRIBUTOR
    48    toddrme2178         1       CONTRIBUTOR
    49        vidartf         1      COLLABORATOR
    50         xmnlab         1       CONTRIBUTOR
    41     ssanderson         1       CONTRIBUTOR
    26       jbweston         1            MEMBER
    28  johnjarmitage         1              NONE
    1    JohanMabille         1       CONTRIBUTOR
    3      SolarisYan         1       CONTRIBUTOR
    4   SylvainCorlay         1            MEMBER
    5      Xarthisius         1       CONTRIBUTOR
    7    abhinavsagar         1       CONTRIBUTOR
    8           afeld         1       CONTRIBUTOR
    9       akchinSTC         1      COLLABORATOR
    10       asreimer         1       CONTRIBUTOR
    11     basnijholt         1       CONTRIBUTOR
    13      blink1073         1            MEMBER
    15   consideRatio         1      COLLABORATOR
    16    damianavila         1            MEMBER
    18   davidanthoff         1       CONTRIBUTOR
    20        ednut15         1       CONTRIBUTOR
    21         eromoe         1       CONTRIBUTOR
    22      fmaussion         1       CONTRIBUTOR
    23        gnestor         1            MEMBER,
    encoding: Fac

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        org       repo username  closed
    0  ipython  ipykernel    minrk       9
    1  ipython    ipython  Carreau      84,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:         org                repo       username  closed
    2   jupyter       docker-stacks        parente       1
    3   jupyter  enterprise_gateway      akchinSTC       2
    4   jupyter  enterprise_gateway    kevin-bates       7
    5   jupyter  enterprise_gateway       lresende       2
    6   jupyter        jupyter-book      choldgraf       6
    7   jupyter      jupyter-sphinx       akhmerov       2
    8   jupyter      jupyter_client  SylvainCorlay       1
    9   jupyter      jupyter_client         rgbkrk       4
    10  jupyter        jupyter_core          minrk       1
    11  jupyter        jupyter_core         rgbkrk       1
    12  jupyter        jupyter_core      takluyver       5
    13  jupyter      jupyter_server      blink1073       1
    14  jupyter      jupyter_server       rolweber       1
    15  jupyter               nb2kg    kevin-bates       1
    16  jupyter           nbconvert          MSeal       6
    17  jupyter           nbconvert       t-makaro       2
    18  jupyter           nbconvert       willingc       1
    19  jupyter              nbdime        vidartf       1
    20  jupyter            nbgrader       jhamrick      24
    21  jupyter            nbviewer      choldgraf       1
    22  jupyter            nbviewer        parente       1
    23  jupyter            notebook       lresende       8
    24  jupyter            notebook          minrk       2
    25  jupyter            notebook      takluyver      22
    26  jupyter         repo2docker        betatim       8
    27  jupyter         repo2docker      choldgraf       1
    28  jupyter         repo2docker          minrk       1
    29  jupyter         repo2docker      yuvipanda       1
    30  jupyter           telemetry     jaipreet-s       2
    31  jupyter           telemetry      yuvipanda       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 30 days',
    width: 1000
  }), Chart({
    data:                 org                 repo       username  closed
    32  jupyter-widgets           ipyleaflet  SylvainCorlay      12
    33  jupyter-widgets           ipyleaflet    martinRenou      36
    34  jupyter-widgets           ipywidgets  SylvainCorlay      20
    35  jupyter-widgets           ipywidgets        vidartf       4
    36  jupyter-widgets            pythreejs        vidartf       8
    37  jupyter-widgets           traittypes  SylvainCorlay       4
    38  jupyter-widgets             tutorial    martinRenou      12
    39  jupyter-widgets             tutorial        mwcraig      52
    40  jupyter-widgets  widget-cookiecutter        vidartf       4,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 30 days',
    width: 1000
  }), Chart({
    data:            org                     repo        username  closed
    41  jupyterhub     alabaster-jupyterhub       choldgraf      12
    42  jupyterhub             batchspawner      mbmilligan      14
    43  jupyterhub                binderhub         betatim      16
    44  jupyterhub                binderhub       choldgraf       4
    45  jupyterhub                binderhub           minrk       2
    46  jupyte

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0   binder-examples                 jupyter-rise       2   
  1           ipython                    ipykernel       3   
  2           ipython                  ipyparallel       2   
  3           ipython                      ipython      18   
  4           jupyter                accessibility       1   
  5           jupyter           enterprise_gateway       2   
  6           jupyter                      jupyter       1   
  7           jupyter                 jupyter-book       1   
  8           jupyter               jupyter-sphinx       1   
  9           jupyter               jupyter_client       1   
  10          jupyter                 jupyter_core       1   
  11          jupyter          jupyter_kernel_test       1   
  12          jupyter               jupyter_server       5   
  13          jupyter                     nbgrader      16   
  14          jupyter                     notebook      23   
  15          jupyter                  repo2docker       6   
  16  jupyter-widgets                   ipyleaflet      12   
  17  jupyter-widgets                   ipywidgets      44   
  18  jupyter-widgets                    pythreejs      12   
  19  jupyter-widgets                     tutorial       4   
  20       jupyterhub                 batchspawner       2   
  21       jupyterhub                    binderhub       8   
  22       jupyterhub                   chartpress       4   
  23       jupyterhub      configurable-http-proxy       4   
  24       jupyterhub                dockerspawner      10   
  25       jupyterhub        firstuseauthenticator       2   
  26       jupyterhub                   helm-chart       2   
  27       jupyterhub       jupyter-rsession-proxy       4   
  28       jupyterhub                   jupyterhub       6   
  29       jupyterhub                  kubespawner       3   
  30       jupyterhub            ldapauthenticator       4   
  31       jupyterhub             ltiauthenticator       2   
  32       jupyterhub          mybinder.org-deploy      12   
  33       jupyterhub                  nbgitpuller       2   
  34       jupyterhub               oauthenticator       2   
  35       jupyterhub                 team-compass       2   
  36       jupyterhub      the-littlest-jupyterhub      13   
  37       jupyterhub                traefik-proxy       2   
  38       jupyterhub                  wrapspawner       4   
  39       jupyterhub       zero-to-jupyterhub-k8s      24   
  40       jupyterlab    extension-cookiecutter-ts       5   
  41       jupyterlab                   jupyterlab     345   
  42       jupyterlab        jupyterlab-commenting      49   
  43       jupyterlab     jupyterlab-data-explorer       5   
  44       jupyterlab  jupyterlab-metadata-service       5   
  45       jupyterlab         jupyterlab-telemetry       5   
  46       jupyterlab               jupyterlab_app       5   
  47       jupyterlab                 team-compass       5   
  48          nteract                    bookstore       4   
  49          nteract                           dx       3   
  50          nteract                     hydrogen       2   
  51          nteract                      nteract      10   
  52          nteract                    papermill       4   
  53          nteract                    scrapbook       2   
  54          nteract                         vdom       1   
  
                                    org/repo  
  0             binder-examples/jupyter-rise  
  1                        ipython/ipykernel  
  2                      ipython/ipyparallel  
  3                          ipython/ipython  
  4                    jupyter/accessibility  
  5               jupyter/enterprise_gateway  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                   jupyter/jupyter-sphinx  
  9                   jupyter/jupyter_cli

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0   binder-examples                     bokeh       2   
  1           ipython                   ipython      18   
  2           ipython                 traitlets       3   
  3           jupyter             docker-stacks       5   
  4           jupyter        enterprise_gateway      10   
  5           jupyter                   jupyter       1   
  6           jupyter              jupyter-book       5   
  7           jupyter            jupyter-sphinx       1   
  8           jupyter              jupyter_core       9   
  9           jupyter            jupyter_server       2   
  10          jupyter                     nb2kg       1   
  11          jupyter                 nbconvert      14   
  12          jupyter                    nbdime       1   
  13          jupyter                  nbgrader      23   
  14          jupyter                  nbviewer       1   
  15          jupyter                  notebook      38   
  16          jupyter               repo2docker       6   
  17  jupyter-widgets                ipyleaflet      40   
  18  jupyter-widgets                ipywidgets      56   
  19  jupyter-widgets                 pythreejs       4   
  20  jupyter-widgets                  tutorial       8   
  21       jupyterhub              batchspawner       2   
  22       jupyterhub                 binderhub      12   
  23       jupyterhub             dockerspawner       2   
  24       jupyterhub                jupyterhub      27   
  25       jupyterhub  jupyterhub-deploy-docker       2   
  26       jupyterhub       mybinder.org-deploy      12   
  27       jupyterhub               nbgitpuller       2   
  28       jupyterhub            oauthenticator       2   
  29       jupyterhub              team-compass      10   
  30       jupyterhub   the-littlest-jupyterhub      18   
  31       jupyterhub             traefik-proxy       4   
  32       jupyterhub               yarnspawner       2   
  33       jupyterhub    zero-to-jupyterhub-k8s      12   
  34       jupyterlab                jupyterlab     464   
  35       jupyterlab  jupyterlab-data-explorer      30   
  36       jupyterlab           jupyterlab-demo      10   
  37       jupyterlab            jupyterlab-git      35   
  38       jupyterlab         jupyterlab_server       6   
  39          nteract                 bookstore       4   
  40          nteract                  hydrogen      73   
  41          nteract                   nteract      20   
  42          nteract                 papermill       5   
  43          nteract                  semiotic       3   
  44          nteract                      vdom       3   
  
                                 org/repo  
  0                 binder-examples/bokeh  
  1                       ipython/ipython  
  2                     ipython/traitlets  
  3                 jupyter/docker-stacks  
  4            jupyter/enterprise_gateway  
  5                       jupyter/jupyter  
  6                  jupyter/jupyter-book  
  7                jupyter/jupyter-sphinx  
  8                  jupyter/jupyter_core  
  9                jupyter/jupyter_server  
  10                        jupyter/nb2kg  
  11                    jupyter/nbconvert  
  12                       jupyter/nbdime  
  13                     jupyter/nbgrader  
  14                     jupyter/nbviewer  
  15                     jupyter/notebook  
  16                  jupyter/repo2docker  
  17           jupyter-widgets/ipyleaflet  
  18           jupyter-widgets/ipywidgets  
  19            jupyter-widgets/pythreejs  
  20             jupyter-widgets/tutorial  
  21              jupyterhub/batchspawner  
  22                 jupyterhub/binderhub  
  23             jupyterhub/dockerspawner  
  24                jupyterhub/jupyterhub  
  25  jupyterhub/jupyterhub-deploy-docker  
  26       jupyterhub/mybinder.org-deploy  
  27               jupyterhub/nbgitpu

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                         repo     kind  count
    40   jupyterlab    extension-cookiecutter-ts  created    5.0
    41   jupyterlab                   jupyterlab  created  345.0
    42   jupyterlab        jupyterlab-commenting  created   49.0
    43   jupyterlab     jupyterlab-data-explorer  created    5.0
    44   jupyterlab  jupyterlab-metadata-service  created    5.0
    45   jupyterlab         jupyterlab-telemetry  created    5.0
    46   jupyterlab               jupyterlab_app  created    5.0
    64   jupyterlab              jupyterlab-demo  created    0.0
    65   jupyterlab               jupyterlab-git  created    0.0
    66   jupyterlab            jupyterlab_server  created    0.0
    108  jupyterlab    extension-cookiecutter-ts   closed    0.0
    109  jupyterlab                   jupyterlab   closed  464.0
    110  jupyterlab        jupyterlab-commenting   closed    0.0
    111  jupyterlab     jupyterlab-data-explorer   closed   30.0
    112  jupyterlab  jupyterlab-metadata-service   closed    0.0
    113  jupyterlab         jupyterlab-telemetry   closed    0.0
    114  jupyterlab               jupyterlab_app   closed    0.0
    132  jupyterlab              jupyterlab-demo   closed   10.0
    133  jupyterlab               jupyterlab-git   closed   35.0
    134  jupyterlab            jupyterlab_server   closed    6.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                 org        repo     kind  count
    16  jupyter-widgets  ipyleaflet  created   12.0
    17  jupyter-widgets  ipywidgets  created   44.0
    18  jupyter-widgets   pythreejs  created   12.0
    19  jupyter-widgets    tutorial  created    4.0
    84  jupyter-widgets  ipyleaflet   closed   40.0
    85  jupyter-widgets  ipywidgets   closed   56.0
    86  jupyter-widgets   pythreejs   closed    4.0
    87  jupyter-widgets    tutorial   closed    8.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org         repo     kind  count
    1    ipython    ipykernel  created    3.0
    2    ipython  ipyparallel  created    2.0
    3    ipython      ipython  created   18.0
    56   ipython    traitlets  created    0.0
    69   ipython    ipykernel   closed    0.0
    70   ipython  ipyparallel   closed    0.0
    71   ipython      ipython   closed   18.0
    124  ipython    traitlets   closed    3.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 da

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                      repo    time_open  \
  0   binder-examples                     bokeh     2.983160   
  1           ipython                   ipython     7.245324   
  2           ipython                 traitlets   106.160822   
  3           jupyter             docker-stacks    11.228750   
  4           jupyter        enterprise_gateway     4.198594   
  5           jupyter                   jupyter     0.550775   
  6           jupyter              jupyter-book     1.179016   
  7           jupyter            jupyter-sphinx     4.944387   
  8           jupyter              jupyter_core   318.299294   
  9           jupyter            jupyter_server    57.569497   
  10          jupyter                     nb2kg    36.362477   
  11          jupyter                 nbconvert  1043.244091   
  12          jupyter                    nbdime    26.252743   
  13          jupyter                  nbgrader   170.229317   
  14          jupyter                  nbviewer    18.274039   
  15          jupyter                  notebook   278.212060   
  16          jupyter               repo2docker    58.781071   
  17  jupyter-widgets                ipyleaflet    13.205660   
  18  jupyter-widgets                ipywidgets     8.697755   
  19  jupyter-widgets                 pythreejs     6.911701   
  20  jupyter-widgets                  tutorial   176.047442   
  21       jupyterhub              batchspawner    87.364086   
  22       jupyterhub                 binderhub     4.980417   
  23       jupyterhub             dockerspawner    27.577431   
  24       jupyterhub                jupyterhub     9.194722   
  25       jupyterhub  jupyterhub-deploy-docker     3.008692   
  26       jupyterhub       mybinder.org-deploy     1.137726   
  27       jupyterhub               nbgitpuller     0.566551   
  28       jupyterhub            oauthenticator   109.493565   
  29       jupyterhub              team-compass    31.010162   
  30       jupyterhub   the-littlest-jupyterhub    16.731308   
  31       jupyterhub             traefik-proxy   113.922025   
  32       jupyterhub               yarnspawner     1.391169   
  33       jupyterhub    zero-to-jupyterhub-k8s     2.553935   
  34       jupyterlab                jupyterlab     3.007164   
  35       jupyterlab  jupyterlab-data-explorer   150.073003   
  36       jupyterlab           jupyterlab-demo    29.906973   
  37       jupyterlab            jupyterlab-git    12.379225   
  38       jupyterlab         jupyterlab_server     8.985532   
  39          nteract                 bookstore     1.319028   
  40          nteract                  hydrogen   400.721215   
  41          nteract                   nteract   331.014682   
  42          nteract                 papermill     9.041065   
  43          nteract                  semiotic    34.916412   
  44          nteract                      vdom   108.073310   
  
                                 org/repo  
  0                 binder-examples/bokeh  
  1                       ipython/ipython  
  2                     ipython/traitlets  
  3                 jupyter/docker-stacks  
  4            jupyter/enterprise_gateway  
  5                       jupyter/jupyter  
  6                  jupyter/jupyter-book  
  7                jupyter/jupyter-sphinx  
  8                  jupyter/jupyter_core  
  9                jupyter/jupyter_server  
  10                        jupyter/nb2kg  
  11                    jupyter/nbconvert  
  12                       jupyter/nbdime  
  13                     jupyter/nbgrader  
  14                     jupyter/nbviewer  
  15                     jupyter/notebook  
  16                  jupyter/repo2docker  
  17           jupyter-widgets/ipyleaflet  
  18           jupyter-widgets/ipywidgets  
  19            jupyter-widgets/pythreejs  
  20             jupyter-widgets/tutorial  
  21              jupyterhub/batchspawner  
  22                 jupyte

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 author  count authorAssociation
    0            choldgraf      6            MEMBER
    1  mathematicalmichael      4              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                    author  count authorAssociation
    1                 Carreau     93            MEMBER
    6          LucianaMarques     36       CONTRIBUTOR
    47                  minrk     24            MEMBER
    20             ccordoba12     18       CONTRIBUTOR
    15                   alok     12       CONTRIBUTOR
    28            drunkwcodes      9              NONE
    51               rmorshea      9      COLLABORATOR
    34               gpotter2      9       CONTRIBUTOR
    54              tacaswell      6       CONTRIBUTOR
    50                 rgbkrk      6            MEMBER
    43                 kmaork      6       CONTRIBUTOR
    42            kevin-bates      6       CONTRIBUTOR
    38               jfhbrook      6              NONE
    30  faultdiagnosistoolbox      6              NONE
    27                 dlukes      6              NONE
    25               dhomeier      6              NONE
    56              ztaylor96      6              NONE
    16                blueyed      6       CONTRIBUTOR
    3           FabioLuporini      6              NONE
    17         brandonwillard      3       CONTRIBUTOR
    5                   Jamim      3       CONTRIBUTOR
    40     jorisvandenbossche      3       CONTRIBUTOR
    41            justgfather      3              NONE
    8                   MSeal      3       CONTRIBUTOR
    7            LunarLanding      3              NONE
    44                kmcminn      3              NONE
    45                 lidd77      3              NONE
    46             michaelaye      3       CONTRIBUTOR
    48                 mwouts      3              NONE
    9                   Mbaey      3              NONE
    49              pintu4all      3              NONE
    4              HamdiTarek      3              NONE
    52                 sizhky      3              NONE
    53               t-makaro      3              NONE
    2                 Chilipp      3       CONTRIBUTOR
    55             yogevyuval      3              NONE
    39              jmansilla      3              NONE
    37                jakevdp      3       CONTRIBUTOR
    18                  cbrnr      3              NONE
    13               YahavFes      3              NONE
    19                cclauss      3              NONE
    21                csheaff      3              NONE
    22                 daharn      3       CONTRIBUTOR
    23             dandavison      3              NONE
    24                dashesy      3              NONE
    14          abaybektursun      3              NONE
    26                dickmao      3              NONE
    12                 Stuj79      3              NONE
    36               ilyaas97      3              NONE
    29         farisachugthai      3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:       author  n_first_responses authorAssociation
    0  choldgraf                  2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:                    author  n_first_responses authorAssociation
    0                 Carreau                 17            MEMBER
    4          LucianaMarques                  7       CONTRIBUTOR
    33                  minrk                  4            MEMBER
    12                blueyed                  2       CONTRIBUTOR
    24               gpotter2                  2       CONTRIBUTOR
    14             ccordoba12                  2       CONTRIBUTOR
    25                hroncok                  1       CONTRIBUTOR
    26                jakevdp                  1       CONTRIBUTOR
    27               jfhbrook                  1              NONE
    28              jmansilla                  1              NONE
    29     jorisvandenbossche                  1       CONTRIBUTOR
    30            justgfather                  1              NONE
    31                kmcminn                  1              NONE
    22           freedomtowin                  1              NONE
    32                 lidd77                  1              NONE
    34              pintu4all                  1              NONE
    35                 rgbkrk                  1            MEMBER
    36               rmorshea                  1      COLLABORATOR
    37                 sizhky                  1              NONE
    38                 stas00                  1              NONE
    39               t-makaro                  1              NONE
    23              gdhameeja                  1              NONE
    20  faultdiagnosistoolbox                  1              NONE
    21           flying-sheep                  1       CONTRIBUTOR
    1           FabioLuporini                  1              NONE
    2              HamdiTarek                  1              NONE
    3                   Jamim                  1       CONTRIBUTOR
    5            LunarLanding                  1              NONE
    6                   MSeal                  1       CONTRIBUTOR
    7                   Mbaey                  1              NONE
    8         MustafaWaheed91                  1              NONE
    9             Stonenumber                  1              NONE
    10               YahavFes                  1              NONE
    11          abaybektursun                  1              NONE
    13                  cbrnr                  1              NONE
    15                csheaff                  1              NONE
    16             dandavison                  1              NONE
    17               dhomeier                  1              NONE
    18                dickmao                  1              NONE
    19                 dlukes                  1              NONE
    40              ztaylor96                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.8

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>